In [1]:
import pandas as pd

pd.set_option('display.max_columns', 100)

In [2]:
marquee= pd.read_json('./account/Marquee.json')
history_1= pd.read_json('./account/StreamingHistory0.json')
history_2= pd.read_json('./account/StreamingHistory1.json')

print(marquee.shape, history_1.shape, history_2.shape)

(2593, 2) (10000, 4) (7589, 4)


In [3]:
history_1.head()

,endTime,artistName,trackName,msPlayed
0,2022-11-08 13:26,Disciple,"Dear X, You Don't Own Me",121303
1,2022-11-08 14:16,Hollywood Undead,Day Of The Dead,99912
2,2022-11-09 01:47,Slipknot,Duality,252613
3,2022-11-09 01:51,All Good Things,For the Glory,279308
4,2022-11-09 01:55,Solence,Animal in Me,5665


In [4]:
history_2.head()

,endTime,artistName,trackName,msPlayed
0,2023-05-14 13:13,Eva Under Fire,Until Forever,183093
1,2023-05-14 13:16,Eva Under Fire,Until Forever,183093
2,2023-05-14 13:19,Eva Under Fire,Until Forever,183093
3,2023-05-14 13:23,Eva Under Fire,Until Forever,183093
4,2023-05-14 13:26,Eva Under Fire,Until Forever,183093


In [5]:
history_2.tail()

,endTime,artistName,trackName,msPlayed
7584,2023-11-08 01:39,Fox Stevenson,Go Like,464
7585,2023-11-08 01:39,Amaranthe,Drop Dead Cynical,0
7586,2023-11-08 01:39,Virtual Riot,Energy Drink,281240
7587,2023-11-08 01:43,Dynazty,Heartless Madness,239866
7588,2023-11-08 01:47,Fox Stevenson,Go Like,233946


In [7]:
history= pd.concat([history_1, history_2]).reset_index()
history.head()

,index,endTime,artistName,trackName,msPlayed
0,0,2022-11-08 13:26,Disciple,"Dear X, You Don't Own Me",121303
1,1,2022-11-08 14:16,Hollywood Undead,Day Of The Dead,99912
2,2,2022-11-09 01:47,Slipknot,Duality,252613
3,3,2022-11-09 01:51,All Good Things,For the Glory,279308
4,4,2022-11-09 01:55,Solence,Animal in Me,5665


In [9]:
history.drop('index', axis= 1, inplace= True)
history.head()

,endTime,artistName,trackName,msPlayed
0,2022-11-08 13:26,Disciple,"Dear X, You Don't Own Me",121303
1,2022-11-08 14:16,Hollywood Undead,Day Of The Dead,99912
2,2022-11-09 01:47,Slipknot,Duality,252613
3,2022-11-09 01:51,All Good Things,For the Glory,279308
4,2022-11-09 01:55,Solence,Animal in Me,5665


In [11]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17589 entries, 0 to 17588
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     17589 non-null  object
 1   artistName  17589 non-null  object
 2   trackName   17589 non-null  object
 3   msPlayed    17589 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 549.8+ KB


In [12]:
# 스트리밍 종료 시간 변환
history['endTime']= pd.to_datetime(history['endTime'])
# 아티스트명+ 곡명
history['fullName']= history.artistName+ ' : '+ history.trackName
# 스트리밍 연월
history['streamingYearMonth']= history.endTime.dt.strftime('%Y-%m')

history.head()

,endTime,artistName,trackName,msPlayed,fullName,streamingYearMonth
0,2022-11-08 13:26:00,Disciple,"Dear X, You Don't Own Me",121303,"Disciple : Dear X, You Don't Own Me",2022-11
1,2022-11-08 14:16:00,Hollywood Undead,Day Of The Dead,99912,Hollywood Undead : Day Of The Dead,2022-11
2,2022-11-09 01:47:00,Slipknot,Duality,252613,Slipknot : Duality,2022-11
3,2022-11-09 01:51:00,All Good Things,For the Glory,279308,All Good Things : For the Glory,2022-11
4,2022-11-09 01:55:00,Solence,Animal in Me,5665,Solence : Animal in Me,2022-11


In [13]:
# 데이터 기간 확인
history.endTime.describe()

C:\Users\1\AppData\Local\Temp\ipykernel_30124\791113630.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  history.endTime.describe()


count                   17589
unique                   9992
top       2023-06-22 07:45:00
freq                       29
first     2022-11-08 13:26:00
last      2023-11-08 01:47:00
Name: endTime, dtype: object

In [14]:
# 곡의 스킵 여부 판단
# 트랙의 러닝타임 정보 필요
# api 활용
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager= SpotifyClientCredentials(client_id= '5e31691277084c049eb5fdee39722645',
                                                     client_secret= '32e599441b624e8197b0380595cbb0bb')

sp= spotipy.Spotify(client_credentials_manager= client_credentials_manager)

In [15]:
# api 활용 트랙의 id 추출
artist_name= []
track_name= []
track_id= []
# 해당 기간에 재생한 음악들
track_history= history[['artistName', 'trackName']].drop_duplicates()

for index in track_history.index :
    # 트랙별 아티스트명, 음악명 일치하는 결과 상위 1개씩만 검색
    result= sp.search(q= [f"artist : {track_history['artistName'][index]}",
                          f"track : {track_history['trackName'][index]}"],
                        type= 'track', limit= 1)

    for i, t in enumerate(result['tracks']['items']) :
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])

df_tracks= pd.DataFrame({'artistName' : artist_name,
                         'trackName' : track_name,
                         'trackId' : track_id})

df_tracks

,artistName,trackName,trackId
0,Disciple,Invisible,63LW4zVVWtrM5cVrJVREM5
1,Hollywood Undead,Undead,5wBLDkxVvclanSitx5jq8e
2,Slipknot,Custer,3GiJq4AQK7324mfIQbpiTf
3,All Good Things,For the Glory,030D3duTUTfJORIycOhZNr
4,Solence,Blackout,4qn2rc3Y91HUSCzC61z6qa
...,...,...,...
2735,Vicetone,Nothing But Love For You,60VLtkNdzSiwESwBDlRFst
2736,Ludwig van Beethoven,"Piano Sonata No. 25 in G Major, Op. 79: Piano ...",2lmoB2oA2ODySWOOIxuADl
2737,Bebe Stockwell,Frosty,3zQ1sEr7c5fo16jROROwuP
2738,Aad Aalberts,Focus,7r9M2GybYSWqLaFI3TjS5i


In [16]:
track_history.shape, df_tracks.shape

((2740, 2), (2740, 3))

In [17]:
# 추출한 곡들의 오디오 특성 추출
# 여기서 러닝타임만 추출 : durationMs
audio_features= []

for track_id in df_tracks.trackId :
    features= sp.audio_features(track_id)
    audio_features.append(features)

df_features= pd.DataFrame(columns= ['danceability', 'energy', 'loudness',
                                      'speechiness', 'acousticness', 'instrumentalness',
                                      'liveness', 'valence', 'tempo', 'id',
                                      'duration_ms', 'time_signature'])

for item in audio_features :
    for feature in item :
        df_features= df_features.append(feature, ignore_index= True)

df_tracks['durationMs']= df_features['duration_ms']
df_tracks

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=777Sc3fVPh7pRPBhMhc4KR:
 Max Retries, reason: too many 429 error responses